<a href="https://colab.research.google.com/github/vincenzomanzoni/alasc/blob/master/02_Pytorch_Forecasting_Example_NBeats_with_Tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch Forecasting | N-Beats

## Install Pytorch Forecasting and import libraries

In [ ]:
!pip install pytorch-forecasting

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_forecasting import TimeSeriesDataSet, Baseline, NBeats
from pytorch_forecasting.data.examples import generate_ar_data
from pytorch_forecasting.metrics import SMAPE

## Dataset generation

In [ ]:
data = generate_ar_data(seasonality=12, timesteps=364, n_series=1, seed=42, trend = 3.0, noise = 0.2)

In [ ]:
data.head()

In [ ]:
plt.plot(data.time_idx, data.value)
plt.title('Generated data')
plt.xlabel('Time index')
plt.ylabel('Value')
plt.show()

In [ ]:
plt.plot(data.value)
plt.xlim(354-80, 354)
plt.title('Generated data')
plt.xlabel('Time index')
plt.ylabel('Value')
plt.show()

## Creation of datasets and dataloaders

In [ ]:
# Create dataset and dataloaders
max_encoder_length = 60
max_prediction_length = 20
batch_size = 16

training_cutoff = data["time_idx"].max() - max_prediction_length

context_length = max_encoder_length
prediction_length = max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="value",
    group_ids=["series"],
    time_varying_unknown_reals=["value"],    
    max_encoder_length=context_length,
    max_prediction_length=prediction_length,    
)
validation = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx=training_cutoff + 1)

train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

## Calculate Baseline Error

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
SMAPE()(baseline_predictions, actuals)

## Train NBeats

In [ ]:
pl.seed_everything(42)

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger_callback = LearningRateMonitor() 

trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    weights_summary="top",
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback, lr_logger_callback],
    limit_train_batches=30,
)

net = NBeats.from_dataset(
    training,
    learning_rate=0.1,
    weight_decay=1e-2,
    widths=[32, 512],
    backcast_loss_ratio=1.0,
)

### Find Optimal Learning Rate

In [ ]:
res = trainer.tuner.lr_find(net, train_dataloader=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5)
fig = res.plot(show=True, suggest=True)
fig.show()

Sometime the red point in the chart does not correspond to the minimim of the loss function.

In [ ]:
print(f"Suggested learning rate: {res.suggestion()}")

In [ ]:
# Look at the above char and put here the learning rate which corresponds
# to the minimum of the function. Usually, 0.1 is a good choice.
net.hparams.learning_rate = 0.1

### Set final parameters before the training

In [ ]:
net.hparams.log_interval = 10
net.hparams.log_val_interval = 1

### Training time

In [ ]:
trainer.fit(
    net,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

## The best model and its performance

In [ ]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = NBeats.load_from_checkpoint(best_model_path)
print(best_model_path)

### Metrics of the best model

In [ ]:
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_model.predict(val_dataloader)
print("MAE: {0:.3}".format((actuals - predictions).abs().mean().item()))
print("MAPE: {0:.3}%".format((100 * (actuals - predictions) / actuals).abs().mean().item()))

In [ ]:
raw_predictions, x = best_model.predict(val_dataloader, mode="raw", return_x=True)
best_model.plot_prediction(x, raw_predictions, add_loss_to_title=True);

In [ ]:
best_model.plot_interpretation(x, raw_predictions, idx=0);

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir lightning_logs